In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from astropy.io import fits
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from pytorch_msssim import ssim, ms_ssim
from tqdm import tqdm
import os
from google.colab import drive
from torchvision.utils import make_grid
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from torch.nn import init
import torch.nn.functional as F

In [ ]:
SAVE_DIR = '/content/drive/MyDrive/EXXA_Results'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, 'training_progress'), exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, 'checkpoints'), exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, 'metrics'), exist_ok=True)

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


#Data Loading and Preprocessing

In [ ]:
class DiskDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.fits')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        with fits.open(file_path) as hdul:
            image = hdul[0].data[0]

        image = (image - np.min(image)) / (np.max(image) - np.min(image))

        if len(image.shape) > 2:
            image = image.squeeze()

        image = image[np.newaxis, :, :]

        if self.transform:
            image = self.transform(image)

        return torch.FloatTensor(image)


#Unsupervised Clustering Model

In [ ]:
class DiskClustering:
    def __init__(self, n_clusters=5):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.scaler = StandardScaler()

    def preprocess_images(self, images):
        # Flatten images and normalize
        flattened = images.reshape(images.shape[0], -1)
        return self.scaler.fit_transform(flattened)

    def fit(self, images):
        processed = self.preprocess_images(images)
        self.kmeans.fit(processed)
        return self

    def predict(self, images):
        processed = self.preprocess_images(images)
        return self.kmeans.predict(processed)

    def visualize_clusters(self, images, labels):
        fig, axes = plt.subplots(2, 5, figsize=(20, 8))
        axes = axes.ravel()

        for i in range(self.n_clusters):
            cluster_images = images[labels == i]
            if len(cluster_images) > 0:
                axes[i].imshow(cluster_images[0], cmap='viridis')
                axes[i].set_title(f'Cluster {i}')
                axes[i].axis('off')

        plt.tight_layout()
        plt.show()


#Autoencoder Model

In [ ]:
class DiskAutoencoder(nn.Module):
    def __init__(self, latent_dim=128):
        super(DiskAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * 75 * 75, latent_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128 * 75 * 75),
            nn.Unflatten(1, (128, 75, 75)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def encode(self, x):
        return self.encoder(x)

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

#Transit Curve Classifier

In [ ]:
class TransitClassifier(nn.Module):
    def __init__(self, input_size=100):
        super(TransitClassifier, self).__init__()

        # Input normalization
        self.input_norm = nn.BatchNorm1d(input_size)

        # Simplified and effective feature extraction
        self.features = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Simple attention
        self.attention = nn.Sequential(
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32, 1),
            nn.Softmax(dim=1)
        )

        # Classifier head
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),
            nn.Linear(32, 1)
        )

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.input_norm(x)
        features = self.features(x)
        attention = self.attention(features)
        weighted_features = features * attention
        return self.classifier(weighted_features)


In [ ]:
ef generate_transit_curves(n_samples=10000, n_points=100):
    curves = np.zeros((n_samples, n_points))
    labels = np.zeros(n_samples)

    for i in range(n_samples):
        t = np.linspace(0, 1, n_points)
        if np.random.random() > 0.5:  # Transit present
            # Clearer transit parameters
            depth = np.random.uniform(0.03, 0.15)  # Deeper transits
            duration = np.random.uniform(0.1, 0.2)  # More consistent duration
            center = np.random.uniform(0.3, 0.7)  # More centered transits

            # Create transit
            transit = np.ones_like(t)
            mask = np.abs(t - center) < duration/2
            transit[mask] = 1 - depth

            # Reduced noise
            noise = np.random.normal(0, 0.001, len(t))  # Less noise
            curve = transit + noise

            curves[i] = curve
            labels[i] = 1
        else:  # No transit
            # Clean non-transit case
            noise = np.random.normal(0, 0.001, len(t))
            trend = np.random.uniform(-0.005, 0.005) * t  # Smaller trends
            curve = 1 + noise + trend

            curves[i] = curve
            labels[i] = 0

        # Normalize
        curves[i] = (curves[i] - np.mean(curves[i])) / np.std(curves[i])

    return curves, labels


#Training Functions

In [ ]:
def visualize_training_progress(model, dataloader, epoch, save_dir=None):
    """Visualize and save training progress"""
    if save_dir is None:
        save_dir = os.path.join(SAVE_DIR, 'training_progress')

    model.eval()
    with torch.no_grad():
        batch = next(iter(dataloader))
        batch = batch.to(device)
        reconstructions = model(batch)
        n = min(8, batch.size(0))
        comparison = torch.cat([batch[:n], reconstructions[:n]])
        grid = make_grid(comparison.cpu(), nrow=n, normalize=True)

        plt.figure(figsize=(12, 4))
        plt.imshow(grid.permute(1, 2, 0))
        plt.axis('off')
        plt.savefig(os.path.join(save_dir, f'epoch_{epoch}.png'))
        plt.close()

In [ ]:
def save_checkpoint(model, optimizer, epoch, loss, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, path)

In [ ]:
def train_autoencoder(model, train_loader, num_epochs=100):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Use SAVE_DIR for checkpoints
    checkpoint_dir = os.path.join(SAVE_DIR, 'checkpoints')

    history = {'train_loss': [], 'val_loss': []}

    # Split data into train and validation
    train_size = int(0.8 * len(train_loader.dataset))
    val_size = len(train_loader.dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(train_loader.dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0

        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            batch = batch.to(device)

            optimizer.zero_grad()
            output = model(batch)
            loss = criterion(output, batch)

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                output = model(batch)
                loss = criterion(output, batch)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        # Update history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)

        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

        # Save checkpoint if validation loss improved
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            save_checkpoint(model, optimizer, epoch, avg_val_loss,
                          os.path.join(checkpoint_dir, 'best_autoencoder.pth'))

        # Save regular checkpoint
        if (epoch + 1) % 10 == 0:
            save_checkpoint(model, optimizer, epoch, avg_train_loss,
                          os.path.join(checkpoint_dir, f'autoencoder_epoch_{epoch+1}.pth'))

        # Visualize progress
        visualize_training_progress(model, train_loader, epoch)

    # Plot and save training history
    plt.figure(figsize=(10, 6))
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(SAVE_DIR, 'autoencoder_training_history.png'))
    plt.close()

    return history




In [ ]:
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch, labels in dataloader:
            batch, labels = batch.to(device), labels.to(device)
            outputs = model(batch)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds).squeeze()
    all_labels = np.array(all_labels)

    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(all_labels, all_preds)
    roc_auc = auc(fpr, tpr)

    # Calculate Precision-Recall curve
    precision, recall, _ = precision_recall_curve(all_labels, all_preds)
    ap = average_precision_score(all_labels, all_preds)

    return {
        'roc_auc': roc_auc,
        'ap': ap,
        'fpr': fpr,
        'tpr': tpr,
        'precision': precision,
        'recall': recall
    }

In [ ]:
def plot_metrics(metrics, save_dir=None):
    if save_dir is None:
        save_dir = os.path.join(SAVE_DIR, 'metrics')


    # Plot ROC curve
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(metrics['fpr'], metrics['tpr'], color='darkorange', lw=2,
             label=f'ROC curve (AUC = {metrics["roc_auc"]:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")

    # Plot Precision-Recall curve
    plt.subplot(1, 2, 2)
    plt.plot(metrics['recall'], metrics['precision'], color='blue', lw=2,
             label=f'PR curve (AP = {metrics["ap"]:.2f})')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower right")

    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'performance_curves.png'))
    plt.close()

In [ ]:
def train_classifier(model, train_loader, val_loader, num_epochs=100):
    criterion = nn.BCEWithLogitsLoss()

    # Better optimizer settings
    optimizer = optim.AdamW(
        model.parameters(),
        lr=0.0003,
        weight_decay=0.001,
        betas=(0.9, 0.999)
    )

    # OneCycleLR scheduler
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.001,
        epochs=num_epochs,
        steps_per_epoch=len(train_loader),
        pct_start=0.3,
        div_factor=25.0,
        final_div_factor=1000.0
    )

    best_val_acc = 0
    patience_counter = 0
    max_patience = 15
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        train_correct = 0
        train_total = 0

        for batch, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            batch, labels = batch.to(device), labels.to(device)

            optimizer.zero_grad()
            output = model(batch)
            loss = criterion(output.squeeze(), labels.float())

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()

            total_train_loss += loss.item()
            predicted = (torch.sigmoid(output) > 0.5).float()
            train_total += labels.size(0)
            train_correct += (predicted.squeeze() == labels).sum().item()

        train_accuracy = 100 * train_correct / train_total
        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch, labels in val_loader:
                batch, labels = batch.to(device), labels.to(device)
                output = model(batch)
                loss = criterion(output.squeeze(), labels.float())

                total_val_loss += loss.item()
                predicted = (torch.sigmoid(output) > 0.5).float()
                val_total += labels.size(0)
                val_correct += (predicted.squeeze() == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        avg_val_loss = total_val_loss / len(val_loader)

        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)

        print(f'Epoch {epoch+1}:')
        print(f'Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_acc': val_accuracy,
            }, os.path.join(SAVE_DIR, 'best_classifier.pth'))
        else:
            patience_counter += 1

        if patience_counter >= max_patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    return history

In [ ]:
if __name__ == "__main__":
    DATA_DIR = '/content/drive/MyDrive/continuum_data_subset'

    # Create datasets and dataloaders
    disk_dataset = DiskDataset(DATA_DIR)
    disk_loader = DataLoader(disk_dataset, batch_size=64, shuffle=True)

    # Initialize models
    clustering_model = DiskClustering(n_clusters=5)
    autoencoder = DiskAutoencoder().to(device)
    classifier = TransitClassifier().to(device)

    # Train models
    print('Training Autoencoder...')
    train_autoencoder(autoencoder, disk_loader)

    print('Training Classifier...')
    # Generate more data for better training
    transit_curves, labels = generate_transit_curves(n_samples=5000)

    # Create datasets
    dataset = torch.utils.data.TensorDataset(
        torch.FloatTensor(transit_curves),
        torch.FloatTensor(labels)
    )

    # Split into train, validation, and test sets
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size

    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size]
    )

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Initialize and train model
    history = train_classifier(classifier, train_loader, val_loader)

    # Evaluate on test set
    classifier.load_state_dict(torch.load(os.path.join(SAVE_DIR, 'best_classifier.pth'))['model_state_dict'])
    test_metrics = evaluate_model(classifier, test_loader, device)
    plot_metrics(test_metrics)

    # Plot training history
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title('Loss History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Val Accuracy')
    plt.title('Accuracy History')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, 'classifier_training_history.png'))
    plt.close()

Training Autoencoder...


Epoch 1/50: 100%|██████████| 4/4 [00:06<00:00,  1.70s/it]


Epoch 1, Train Loss: 0.0741, Val Loss: 0.0151


Epoch 2/50: 100%|██████████| 4/4 [00:03<00:00,  1.10it/s]


Epoch 2, Train Loss: 0.0115, Val Loss: 0.0150


...

Epoch 25:
Train Loss: 0.5185, Train Acc: 79.00%
Val Loss: 0.5089, Val Acc: 67.33%


Epoch 26/100: 100%|██████████| 110/110 [00:01<00:00, 104.06it/s]


Epoch 26:
Train Loss: 0.5183, Train Acc: 78.94%
Val Loss: 0.5086, Val Acc: 66.40%


Epoch 27/100: 100%|██████████| 110/110 [00:01<00:00, 105.04it/s]


Epoch 27:
Train Loss: 0.5166, Train Acc: 78.71%
Val Loss: 0.5093, Val Acc: 74.27%
Early stopping triggered at epoch 27
